In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np

In [2]:
tf.keras.backend.clear_session()

In [3]:
SEQUENCE_LENGTH = 60
LSTM_SIZE = 128

In [4]:
embedding_matrix = np.load('./sentiment140_embedding.npy')

In [5]:
# not-exists word

embedding_matrix = np.pad(embedding_matrix, ((0,0),(0,1)), 'constant', constant_values=0)
embedding_matrix[-1][-1] = 1

In [6]:
tf.keras.backend.clear_session()
inputs = tf.keras.layers.Input(shape=(SEQUENCE_LENGTH,))
x = tf.keras.layers.Embedding(embedding_matrix.shape[0],embedding_matrix.shape[1],weights=[embedding_matrix],trainable=False,mask_zero=True)(inputs)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(LSTM_SIZE, return_sequences=True))(x)
# x = tf.keras.layers.SpatialDropout1D(0.2)(x)
x = tf.keras.layers.MultiHeadAttention(num_heads=4, key_dim=200)(x, x)
x = tf.keras.layers.Dropout(0.15)(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(32, activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.15)(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs, name='sentiment140_init_model')
model.summary()

2022-03-20 19:54:28.824304: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-20 19:54:30.410461: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2300 MB memory:  -> device: 0, name: NVIDIA Tesla M40, pci bus id: 0000:08:00.0, compute capability: 5.2
2022-03-20 19:54:30.411268: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 208 MB memory:  -> device: 1, name: NVIDIA Tesla M40, pci bus id: 0000:09:00.0, compute capability: 5.2


Model: "sentiment140_init_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 60)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 60, 301)      8153187     ['input_1[0][0]']                
                                                                                                  
 bidirectional (Bidirectional)  (None, 60, 256)      440320      ['embedding[0][0]']              
                                                                                                  
 multi_head_attention (MultiHea  (None, 60, 256)     821856      ['bidirectional[0][0]',          
 dAttention)                                                      'bidirecti

In [7]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    optimizer=tfa.optimizers.AdamW(weight_decay=1e-4),
    metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0.5)]
)

In [8]:
model.save('./sentiment140_init_model.h5')